Homework 2 - Classification
Name: <
insert name here>
Remember that you are encouraged to discuss the problems with your instructors and classmates, but you must write all code and solutions on your own.

The rules to be followed for the assignment are:

Do NOT load additional packages beyond what we've shared in the cells below.
Some problems with code may be autograded. If we provide a function or class API do not change it.
Do not change the location of the data or data directory. Use only relative paths to access the data.

In [ ]:
import argparse
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from collections import defaultdict

[10 points] Problem 1 - Building a Decision Tree¶
A sample dataset has been provided to you in the './data/dataset.csv' path. Here are the attributes for the dataset. Use this dataset to test your functions.

Age - ["<=30", "31-40", ">40"]
Income - ["low", "medium", "high"]
Student - ["no", "yes"]
Credit Rating - ["fair", "excellent"]
Loan - ["no", "yes"]
Note:

A sample dataset to test your code has been provided in the location "data/dataset.csv". Please maintain this as it would be necessary while grading.
Do not change the variable names of the returned values.
After calculating each of those values, assign them to the corresponding value that is being returned.
The "Loan" attribute should be used as the target variable while making calculations for your decision tree.

In [ ]:
import math
import pandas as pd

def information_gain_target(dataset_file):
    df = pd.read_csv(dataset_file)
    
    # Calculate the entropy of the entire dataset based on the "Loan" attribute
    total_instances = len(df)
    loan_counts = df['Loan'].value_counts()
    entropy_loan = 0
    for count in loan_counts:
        p = count / total_instances
        entropy_loan -= p * math.log2(p)
    
    # Calculate the weighted average entropy for each attribute (Age, Income, Student, Credit Rating)
    attributes = ['Age', 'Income', 'Student', 'Credit Rating']
    weighted_entropy = 0
    for attribute in attributes:
        attribute_values = df[attribute].unique()
        entropy_attribute = 0
        for value in attribute_values:
            subset = df[df[attribute] == value]
            subset_instances = len(subset)
            subset_loan_counts = subset['Loan'].value_counts()
            subset_entropy = 0
            for sub_count in subset_loan_counts:
                p_sub = sub_count / subset_instances
                subset_entropy -= p_sub * math.log2(p_sub)
            entropy_attribute += (subset_instances / total_instances) * subset_entropy
        
        weighted_entropy += entropy_attribute
    
    # Calculate information gain by subtracting the weighted average entropy from the entropy of the entire dataset
    ig_loan = entropy_loan
    
    return ig_loan

In [ ]:
# This cell has visible test cases that you can run to see if you are on the right track!
# Note: hidden tests will also be applied on other datasets for final grading.

ig_loan = information_gain_target('./data/dataset.csv')
ig_loan_expected = 0.9798687566511528

print(f'The expected ig_loan value for the given dataset is: {ig_loan_expected}')
print(f'Your ig_loan value is: {ig_loan}')

try:
    np.testing.assert_allclose(ig_loan, ig_loan_expected, rtol=0.001, atol=0.001)
    print("Visible tests passed!")
except:
    print("Visible tests failed!")

Grade cell: cell-831583db0fac12a6Score: 5.0 / 5.0 (Top)
# This cell has hidden test cases that will run after you submit your assignment. 

Hidden Tests Redacted
Congratulations! All test cases in this cell passed.

In [ ]:
import math

def information_gain(p_count_yes, p_count_no):
    # Ensure that there are no division by zero errors
    if p_count_yes == 0 or p_count_no == 0:
        return 0
    
    # Calculate the probabilities
    total_count = p_count_yes + p_count_no
    p_yes = p_count_yes / total_count
    p_no = p_count_no / total_count
    
    # Calculate information gain
    ig = -p_yes * math.log2(p_yes) - p_no * math.log2(p_no)
    

In [ ]:
import operator
import pandas as pd
import math

attribute_values = {
    "Age": ["<=30", "31-40", ">40"],
    "Income": ["low", "medium", "high"],
    "Student": ["yes", "no"],
    "Credit Rating": ["fair", "excellent"]
}

attributes = ["Age", "Income", "Student", "Credit Rating"]

def information_gain_attributes(dataset_file, ig_loan, attributes, attribute_values):
    results = {
        "ig_attributes": {
            "Age": 0,
            "Income": 0,
            "Student": 0,
            "Credit Rating": 0
        },
        "best_attribute": ""
    }
    
    df = pd.read_csv(dataset_file)
    total_instances = len(df)
    
    for attribute in attributes:
        ig_attribute = 0
        
        for att_value in attribute_values[attribute]:
            subset = df[df[attribute] == att_value]
            subset_instances = len(subset)
            subset_loan_counts = subset['Loan'].value_counts()
            subset_entropy = 0
            
            for count in subset_loan_counts:
                p = count / subset_instances
                subset_entropy -= p * math.log2(p)
            
            ig_attribute += (subset_instances / total_instances) * subset_entropy
        
        results["ig_attributes"][attribute] = ig_loan - ig_attribute
    
    results["best_attribute"] = max(results["ig_attributes"].items(), key=operator.itemgetter(1))[0]
    return results

In [ ]:
# This cell has visible test cases that you can run to see if you are on the right track!
# Note: hidden tests will also be applied on other datasets for final grading.

import pprint
pp = pprint.PrettyPrinter(depth=4)
ig_loan_expected = 0.9798687566511528

attribute_values = {
    "Age": ["<=30", "31-40", ">40"],
    "Income": ["low", "medium", "high"],
    "Student": ["yes", "no"],
    "Credit Rating": ["fair", "excellent"]
}

attributes = ["Age", "Income", "Student", "Credit Rating"]

results = information_gain_attributes("./data/dataset.csv", ig_loan_expected, attributes, attribute_values)

results_expected = {'ig_attributes': {'Age': 0.2419726756283742, 'Income': 0.012398717114751934, 'Student': 0.19570962879973097, 'Credit Rating': 0.07181901063117269}, 'best_attribute': 'Age'}

print(f'The expected results value for the given dataset is:')
pp.pprint(results_expected)
print(f'Your results value is:')
pp.pprint(results)

try:
    x = pd.Series(results["ig_attributes"])
    y = pd.Series(results_expected["ig_attributes"])
    pd.testing.assert_series_equal(x, y, check_less_precise=3)
    assert results["best_attribute"] == results_expected["best_attribute"]
    print("Visible tests passed!")
except:
    print("Visible tests failed!")

Grade cell: cell-dd2742d4c14f088eScore: 5.0 / 5.0 (Top)
# This cell has hidden test cases that will run after you submit your assignment. 

Hidden Tests Redacted
Congratulations! All test cases in this cell passed.

[10 points] Problem 2 - Building a Naive Bayes Classifier¶
A sample dataset has been provided to you in the './data/dataset.csv' path. Here are the attributes for the dataset. Use this dataset to test your functions.

Age - ["<=30", "31-40", ">40"]
Income - ["low", "medium", "high"]
Student - ["no", "yes"]
Credit Rating - ["fair", "excellent"]
Loan - ["no", "yes"]
Note:

A sample dataset to test your code has been provided in the location "data/dataset.csv". Please maintain this as it would be necessary while grading.
Do not change the variable names of the returned values.
After calculating each of those values, assign them to the corresponding value that is being returned.
The "Loan" attribute should be used as the target variable while making calculations for your naive bayes classifier.

In [ ]:
from collections import defaultdict
import pandas as pd

def naive_bayes(dataset_file, attributes, attribute_values):
    probabilities = {
        "Age": {"<=30": {"yes": 0, "no": 0}, "31-40": {"yes": 0, "no": 0}, ">40": {"yes": 0, "no": 0}},
        "Income": {"low": {"yes": 0, "no": 0}, "medium": {"yes": 0, "no": 0}, "high": {"yes": 0, "no": 0}},
        "Student": {"yes": {"yes": 0, "no": 0}, "no": {"yes": 0, "no": 0}},
        "Credit Rating": {"fair": {"yes": 0, "no": 0}, "excellent": {"yes": 0, "no": 0}},
        "Loan": {"yes": 0, "no": 0}
    }
    
    df = pd.read_csv(dataset_file)
    total_instances = len(df)
    
    loan_counts = df["Loan"].value_counts()
    probabilities["Loan"]["yes"] = loan_counts.get("yes", 0) / total_instances
    probabilities["Loan"]["no"] = loan_counts.get("no", 0) / total_instances
    
    for attribute in attributes:
        for att_value in attribute_values[attribute]:
            # Count the occurrences of the attribute value for "yes" and "no" outcomes
            att_value_counts = df[df[attribute] == att_value]["Loan"].value_counts()
            
            # Update the probabilities dictionary with the counts
            probabilities[attribute][att_value]["yes"] = att_value_counts.get("yes", 0) / loan_counts.get("yes", 1)
            probabilities[attribute][att_value]["no"] = att_value_counts.get("no", 0) / loan_counts.get("no", 1)
    
    return probabilities

In [ ]:
# This cell has visible test cases that you can run to see if you are on the right track!
# Note: hidden tests will also be applied on other datasets for final grading.

import pprint
pp = pprint.PrettyPrinter(depth=6)

attribute_values = {
    "Age": ["<=30", "31-40", ">40"],
    "Income": ["low", "medium", "high"],
    "Student": ["yes", "no"],
    "Credit Rating": ["fair", "excellent"]
}

attributes = ["Age", "Income", "Student", "Credit Rating"]

probabilities = naive_bayes("./data/dataset.csv", attributes, attribute_values)

probabilities_expected = {'Age': {'<=30': {'yes': 0.2857142857142857, 'no': 0.6},
  '31-40': {'yes': 0.42857142857142855, 'no': 0.0},
  '>40': {'yes': 0.2857142857142857, 'no': 0.4}},
 'Income': {'low': {'yes': 0.2857142857142857, 'no': 0.2},
  'medium': {'yes': 0.42857142857142855, 'no': 0.4},
  'high': {'yes': 0.2857142857142857, 'no': 0.4}},
 'Student': {'yes': {'yes': 0.7142857142857143, 'no': 0.2},
  'no': {'yes': 0.2857142857142857, 'no': 0.8}},
 'Credit Rating': {'fair': {'yes': 0.7142857142857143, 'no': 0.4},
  'excellent': {'yes': 0.2857142857142857, 'no': 0.6}},
 'Loan': {'yes': 0.5833333333333334, 'no': 0.4166666666666667}}

print(f'Your probabilities value is:')
pp.pprint(probabilities)
print(f'\nThe expected probabilities value for the given dataset is:')
pp.pprint(probabilities_expected)

try:
    for i in attributes:
        for j in attribute_values[i]:
            for k in ["yes", "no"]:
                np.testing.assert_allclose(probabilities[i][j][k], probabilities_expected[i][j][k], rtol=0.001, atol=0.001)
    print("Visible tests passed!")
except:
    print("Visible tests failed!")


Grade cell: cell-1029d20562b90189Score: 10.0 / 10.0 (Top)
# This cell has hidden test cases that will run after you submit your assignment. 

Hidden Tests Redacted
Congratulations! All test cases in this cell passed.